# Tutorial 1 <br /> <br /> Solve Dantzig's Transport Problem using the *ix modeling platform* (ixmp4)

<img style="float: right; height: 80px;" src="_static/python.png">

## Aim and scope of the tutorial

This tutorial takes you through the steps to import the data for a very simple optimization model
and solve it using the **ixmp4**-GAMS interface.

We use Dantzig's transport problem, which is also used as the standard GAMS tutorial.
This problem finds a least cost shipping schedule that meets requirements at markets and supplies at factories.

If you are not familiar with GAMS, please take a minute to look at the [transport.gms](transport.gms) code.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

> This formulation is described in detail in:  
> Rosenthal, R E, Chapter 2: A GAMS Tutorial.  
> In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.

> see http://www.gams.com/mccarl/trnsport.gms

## Tutorial outline

The steps in the tutorial are the following:

0. Launch an **ixmp4.Platform** instance and initialize a new **ixmp4.Run**
0. Define the **sets and parameters** in the scenario and save the data to the platform
0. Initialize **variables and equations** to import the solution from GAMS
0. Call GAMS to **solve the scenario** (export to GAMS input gdx, execute, read solution from output gdx)
0. Display the **solution** (variables and equation)

## Launching the *Platform* and initializing a new *Run*

A **Platform** is the connection to the database that holds all data and relevant additional information.

A **Run** is an object that holds all relevant information for one quantification of a scenario.  
A run is identified by a model name, a scenario name and a version number (assigned automatically).

In [1]:
import pandas as pd
import ixmp4

You have to *register a new local database* before you can run the tutorial. 

Run the following in the command-line:
```
ixmp4 platforms add sqlite-test
```

You can then check if the database was successfully created by running
```
ixmp4 platforms list
```

After creating the database, you can connect to it via an **ixmp4.Platform** instance.

In [2]:
mp = ixmp4.Platform("sqlite-test")

Now, we initialize a new **ixmp4.Run** in the database. This is done by using the argument *version="new"*.

In [3]:
run = mp.Run(model="transport problem", scenario="standard", version="new")

## Defining the *IndexSets*

An **IndexSet** defines a named list of elements. These IndexSets can be used for "indexed assignment" of parameters, variables and equations.  
In database-lingo, a column of a parameter can be "foreign-keyed" onto an IndexSet.

Below, we first show the data as they would be written in the GAMS tutorial ([transport.gms](transport.gms) in this folder).  

We now initialize these sets and assign the elements.

In [4]:
i = run.optimization.IndexSet("i")

In [5]:
i.add(["seattle", "san-diego"])

We can display the elements of **IndexSet i** as a Python list.

In [6]:
i.elements

['seattle', 'san-diego']

For simplicity, the steps of creating an **IndexSet** and assigning elements can be done in one line.

In [7]:
run.optimization.IndexSet("j").add(["new-york", "chicago", "topeka"])

## Assigning the *Parameters*

Next, we define the parameters *capacity* and *demand*. The parameters are assigned on the IndexSets *i* and *j*, respectively.

In [ ]:
# capacity of plant i in cases                                                                                                                                                           
# add parameter elements one-by-one (string and value)                                                                                                                                   
scen.init_par("a", idx_sets="i")
scen.add_par("a", "seattle", 350, "cases")
scen.add_par("a", "san-diego", 600, "cases")

# demand at market j in cases                                                                                                                                                            
# add parameter elements as dataframe (with index names)                                                                                                                                 
scen.init_par("b", idx_sets="j")
b_data = [
    {'j': "new-york", 'value': 325, 'unit': "cases"},
    {'j': "chicago",  'value': 300, 'unit': "cases"},
    {'j': "topeka",   'value': 275, 'unit': "cases"}
]
b = pd.DataFrame(b_data)
scen.add_par("b", b)

In [ ]:
scen.par('b')

In [ ]:
# distance in thousands of miles                                                                                                                                                         
scen.init_par("d", idx_sets=["i", "j"])
# add more parameter elements as dataframe by index names                                                                                                                                
d_data = [
    {'i': "seattle", 'j': "new-york", 'value': 2.5, 'unit': "km"},
    {'i': "seattle", 'j': "chicago", 'value': 1.7, 'unit': "km"},
    {'i': "seattle", 'j': "topeka", 'value': 1.8, 'unit': "km"},
    {'i': "san-diego", 'j': "new-york", 'value': 2.5, 'unit': "km"},
]
d = pd.DataFrame(d_data)
scen.add_par("d", d)

# add other parameter elements as key list, value, unit
scen.add_par("d", ["san-diego", "chicago"], 1.8, "km")
scen.add_par("d", ["san-diego", "topeka"], 1.4, "km")

In [ ]:
# cost per case per 1000 miles                                                                                                                                                           
# initialize scalar with a value and a unit (and optionally a comment)                                                                                                                   
scen.init_scalar("f", 90.0, "USD/km")

### Committing the scenario to the ixmp database instance

In [ ]:
# commit new scenario to the database
# no changes can then be made to the scenario data until a check-out is performed
comment = "importing Dantzig's transport problem for illustration"
comment += " and testing of the Python interface using a generic datastructure"                                                                                                                                                                            
scen.commit(comment)      

# set this new scenario as the default version for the model/scenario name
scen.set_as_default()

### Defining variables and equations in the scenario

The levels and marginals of these variables and equations will be imported to the scenario when reading the gdx solution file.

In [ ]:
# perform a check_out to make further changes
scen.check_out()

# initialize the decision variables and equations
scen.init_var("z", None, None)
scen.init_var("x", idx_sets=["i", "j"])
scen.init_equ("demand", idx_sets=["j"])

# commit changes to the scenario (save changes in ixmp database instance)
change_comment = "initialize the model variables and equations"
scen.commit(change_comment)

### Solve the scenario

The ``solve()`` function exports the scenario to a GAMS gdx file, executes GAMS, and then imports the solution from an output GAMS gdx file to the database.

For the model equations and the GAMS workflow (reading the data from gdx, solving the model, writing the results to gdx), see ``transport_ixmp.gms``.

In [ ]:
scen.solve(model='dantzig')

### Display and analyze the results

In [ ]:
# display the objective value of the solution
scen.var("z")

In [ ]:
# display the quantities transported from canning plants to demand locations
scen.var("x")

In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints
scen.equ("demand")

### Close the database connection of the ix modeling platform

Closing the database connection is recommended when working with the local file-based database, i.e., ``dbtype='HSQLDB'``.
This command closes the database files and removes temporary data. This is necessary so that other notebooks or ``ixmp`` instances can access the database file, or so that the database files can be copied to a different folder or drive.

In [ ]:
# close the connection of the platform instance to the local ixmp database files
mp.close_db()